# Healthcare Data Exploration and Principal Component Analysis Tutorial

**Author:** Dr. Priya Lakshmi Narayanan  
**Institute:** Institute of Cancer Research

This tutorial demonstrates practical approaches to working with healthcare data, handling missing data, detecting artifacts, and applying PCA for dimensionality reduction.

## 1. Setup and Data Generation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")

## 2. Electronic Health Record (EHR) Data Simulation

In [ ]:
def generate_ehr_data(n_patients=1000, n_features=50):
    """
    Generate synthetic EHR data with various feature types
    
    Parameters:
    -----------
    n_patients : int
        Number of patient records
    n_features : int
        Number of clinical features
    
    Returns:
    --------
    df : pandas DataFrame
        Synthetic EHR data
    """
    # Patient demographics
    age = np.random.normal(55, 15, n_patients).clip(18, 95)
    gender = np.random.choice(['M', 'F'], n_patients)
    
    # Vital signs with realistic distributions
    systolic_bp = np.random.normal(130, 20, n_patients).clip(90, 200)
    diastolic_bp = np.random.normal(80, 12, n_patients).clip(60, 120)
    heart_rate = np.random.normal(75, 12, n_patients).clip(50, 120)
    temperature = np.random.normal(36.8, 0.5, n_patients).clip(35.5, 39.5)
    
    # Laboratory values
    glucose = np.random.gamma(2, 50, n_patients).clip(60, 400)
    hemoglobin = np.random.normal(14, 2, n_patients).clip(8, 18)
    wbc = np.random.normal(7.5, 2.5, n_patients).clip(3, 15)
    platelets = np.random.normal(250, 60, n_patients).clip(100, 450)
    creatinine = np.random.gamma(2, 0.5, n_patients).clip(0.5, 3.0)
    
    # Biomarkers
    cholesterol = np.random.normal(200, 40, n_patients).clip(120, 350)
    hdl = np.random.normal(50, 15, n_patients).clip(20, 100)
    ldl = np.random.normal(130, 35, n_patients).clip(50, 250)
    triglycerides = np.random.gamma(2, 75, n_patients).clip(40, 400)
    
    # Create DataFrame
    data = {
        'patient_id': range(1, n_patients + 1),
        'age': age,
        'gender': gender,
        'systolic_bp': systolic_bp,
        'diastolic_bp': diastolic_bp,
        'heart_rate': heart_rate,
        'temperature': temperature,
        'glucose': glucose,
        'hemoglobin': hemoglobin,
        'wbc_count': wbc,
        'platelet_count': platelets,
        'creatinine': creatinine,
        'total_cholesterol': cholesterol,
        'hdl_cholesterol': hdl,
        'ldl_cholesterol': ldl,
        'triglycerides': triglycerides,
    }
    
    # Add additional random features to reach n_features
    for i in range(16, n_features):
        data[f'feature_{i}'] = np.random.randn(n_patients)
    
    df = pd.DataFrame(data)
    
    # Add some correlations for realism
    bmi = np.random.normal(26, 5, n_patients).clip(15, 45)
    df['bmi'] = bmi
    df['systolic_bp'] += bmi * 0.5
    df['glucose'] += bmi * 2
    df['systolic_bp'] += age * 0.3
    df['creatinine'] += (age - 40) * 0.01
    
    return df

# Generate data
ehr_data = generate_ehr_data(n_patients=500, n_features=30)

print(f"Generated EHR data shape: {ehr_data.shape}")
print("\nFirst few rows:")
display(ehr_data.head())

print("\nBasic statistics:")
display(ehr_data.describe())

## 3. Exploratory Data Analysis (EDA)

In [ ]:
# Distribution visualization
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Distribution of Key Clinical Variables', fontsize=16, fontweight='bold')

clinical_vars = ['age', 'systolic_bp', 'glucose', 'hemoglobin', 'creatinine', 'bmi']

for idx, var in enumerate(clinical_vars):
    ax = axes[idx // 3, idx % 3]
    ax.hist(ehr_data[var], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
    ax.set_title(f'{var.replace("_", " ").title()}', fontweight='bold')
    ax.set_xlabel('Value')
    ax.set_ylabel('Frequency')
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Missing Data: Introduction and Mechanisms

In [ ]:
def introduce_missing_data(df, missing_patterns):
    """
    Introduce missing data with different mechanisms
    """
    df_missing = df.copy()
    
    for pattern in missing_patterns:
        feature = pattern['feature']
        mechanism = pattern['mechanism']
        rate = pattern['rate']
        
        n = len(df_missing)
        
        if mechanism == 'MCAR':
            missing_mask = np.random.rand(n) < rate
            df_missing.loc[missing_mask, feature] = np.nan
            
        elif mechanism == 'MAR':
            condition_var = pattern.get('condition_var', 'age')
            threshold = df_missing[condition_var].median()
            prob = np.where(df_missing[condition_var] > threshold, rate * 2, rate * 0.5)
            missing_mask = np.random.rand(n) < prob
            df_missing.loc[missing_mask, feature] = np.nan
            
        elif mechanism == 'MNAR':
            threshold = df_missing[feature].quantile(0.75)
            prob = np.where(df_missing[feature] > threshold, rate * 3, rate * 0.3)
            missing_mask = np.random.rand(n) < prob
            df_missing.loc[missing_mask, feature] = np.nan
    
    return df_missing

# Define missing data patterns
missing_patterns = [
    {'feature': 'glucose', 'mechanism': 'MCAR', 'rate': 0.10},
    {'feature': 'hemoglobin', 'mechanism': 'MAR', 'rate': 0.15, 'condition_var': 'age'},
    {'feature': 'creatinine', 'mechanism': 'MNAR', 'rate': 0.12},
    {'feature': 'ldl_cholesterol', 'mechanism': 'MCAR', 'rate': 0.08},
    {'feature': 'bmi', 'mechanism': 'MAR', 'rate': 0.10, 'condition_var': 'age'},
]

ehr_missing = introduce_missing_data(ehr_data, missing_patterns)

# Visualize missing data patterns
plt.figure(figsize=(12, 6))
missing_matrix = ehr_missing.isnull()
plt.imshow(missing_matrix.iloc[:100, :20].T, cmap='RdYlGn_r', aspect='auto', interpolation='none')
plt.colorbar(label='Missing (1) vs Present (0)')
plt.xlabel('Patient Index')
plt.ylabel('Features')
plt.title('Missing Data Pattern Visualization', fontweight='bold')
plt.tight_layout()
plt.show()

# Missing data summary
print("Missing Data Summary:")
print("=" * 50)
missing_summary = pd.DataFrame({
    'Feature': ehr_missing.columns,
    'Missing_Count': ehr_missing.isnull().sum(),
    'Missing_Percentage': (ehr_missing.isnull().sum() / len(ehr_missing) * 100).round(2)
})
missing_summary = missing_summary[missing_summary['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)
display(missing_summary)

## 5. Handling Missing Data: Multiple Approaches

In [ ]:
# Prepare numeric data for imputation
numeric_cols = ehr_missing.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols.remove('patient_id')
X_missing = ehr_missing[numeric_cols].copy()

print(f"Working with {len(numeric_cols)} numeric features")
print(f"Total missing values: {X_missing.isnull().sum().sum()}")

In [ ]:
# Method 1: Mean Imputation
mean_imputer = SimpleImputer(strategy='mean')
X_mean_imputed = pd.DataFrame(mean_imputer.fit_transform(X_missing), columns=numeric_cols)
print("Mean Imputation - Remaining missing values:", X_mean_imputed.isnull().sum().sum())

# Method 2: Median Imputation
median_imputer = SimpleImputer(strategy='median')
X_median_imputed = pd.DataFrame(median_imputer.fit_transform(X_missing), columns=numeric_cols)
print("Median Imputation - Remaining missing values:", X_median_imputed.isnull().sum().sum())

# Method 3: KNN Imputation
knn_imputer = KNNImputer(n_neighbors=5)
X_knn_imputed = pd.DataFrame(knn_imputer.fit_transform(X_missing), columns=numeric_cols)
print("KNN Imputation - Remaining missing values:", X_knn_imputed.isnull().sum().sum())

# Method 4: MICE
mice_imputer = IterativeImputer(random_state=42, max_iter=10)
X_mice_imputed = pd.DataFrame(mice_imputer.fit_transform(X_missing), columns=numeric_cols)
print("MICE Imputation - Remaining missing values:", X_mice_imputed.isnull().sum().sum())

In [ ]:
# Compare imputation methods - glucose variable
glucose_original = ehr_data['glucose']
glucose_missing_mask = ehr_missing['glucose'].isnull()

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
fig.suptitle('Comparison of Imputation Methods: Glucose (MCAR)', fontsize=14, fontweight='bold')

axes[0, 0].hist(glucose_original, bins=30, alpha=0.7, color='green', edgecolor='black')
axes[0, 0].set_title('Original Data')
axes[0, 0].set_xlabel('Glucose (mg/dL)')

axes[0, 1].hist(ehr_missing['glucose'].dropna(), bins=30, alpha=0.7, color='orange', edgecolor='black')
axes[0, 1].set_title(f'With Missing ({glucose_missing_mask.sum()} values)')

axes[0, 2].hist(X_mean_imputed['glucose'], bins=30, alpha=0.7, color='blue', edgecolor='black')
axes[0, 2].set_title('Mean Imputation')

axes[1, 0].hist(X_median_imputed['glucose'], bins=30, alpha=0.7, color='purple', edgecolor='black')
axes[1, 0].set_title('Median Imputation')

axes[1, 1].hist(X_knn_imputed['glucose'], bins=30, alpha=0.7, color='teal', edgecolor='black')
axes[1, 1].set_title('KNN Imputation')

axes[1, 2].hist(X_mice_imputed['glucose'], bins=30, alpha=0.7, color='brown', edgecolor='black')
axes[1, 2].set_title('MICE Imputation')

plt.tight_layout()
plt.show()

# Statistical comparison
print("\nGlucose Statistics Comparison:")
comparison_df = pd.DataFrame({
    'Method': ['Original', 'With Missing', 'Mean', 'Median', 'KNN', 'MICE'],
    'Mean': [
        glucose_original.mean(),
        ehr_missing['glucose'].mean(),
        X_mean_imputed['glucose'].mean(),
        X_median_imputed['glucose'].mean(),
        X_knn_imputed['glucose'].mean(),
        X_mice_imputed['glucose'].mean()
    ],
    'Std': [
        glucose_original.std(),
        ehr_missing['glucose'].std(),
        X_mean_imputed['glucose'].std(),
        X_median_imputed['glucose'].std(),
        X_knn_imputed['glucose'].std(),
        X_mice_imputed['glucose'].std()
    ]
})
display(comparison_df)

## 6. PCA Step 1: Standardization

In [ ]:
# Use mean-imputed data for PCA
pca_data = X_mean_imputed.copy()

print("="*70)
print("STEP 1: STANDARDIZATION")
print("="*70)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(pca_data)
X_scaled_df = pd.DataFrame(X_scaled, columns=pca_data.columns)

print(f"\nOriginal data - Mean: {pca_data.mean().mean():.2f}, Std: {pca_data.std().mean():.2f}")
print(f"Scaled data - Mean: {X_scaled_df.mean().mean():.6f}, Std: {X_scaled_df.std().mean():.6f}")

# Visualize standardization effect
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].boxplot([pca_data.iloc[:, i] for i in range(min(10, len(pca_data.columns)))],
                labels=[col[:8] for col in pca_data.columns[:10]])
axes[0].set_title('Before Standardization', fontweight='bold')
axes[0].tick_params(axis='x', rotation=45)

axes[1].boxplot([X_scaled_df.iloc[:, i] for i in range(min(10, len(X_scaled_df.columns)))],
                labels=[col[:8] for col in X_scaled_df.columns[:10]])
axes[1].set_title('After Standardization', fontweight='bold')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 7. PCA Step 2: Compute Covariance Matrix

In [ ]:
print("="*70)
print("STEP 2: COVARIANCE MATRIX")
print("="*70)

# Compute covariance matrix
cov_matrix_np = np.cov(X_scaled.T)

print(f"\nCovariance matrix shape: {cov_matrix_np.shape}")
print(f"Is symmetric? {np.allclose(cov_matrix_np, cov_matrix_np.T)}")

# Visualize covariance matrix
plt.figure(figsize=(10, 8))
plt.imshow(cov_matrix_np, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
plt.colorbar(label='Covariance')
plt.title('Covariance Matrix Heatmap', fontweight='bold', fontsize=14)
plt.xlabel('Feature Index')
plt.ylabel('Feature Index')
plt.tight_layout()
plt.show()

## 8. PCA Step 3: Eigendecomposition

In [ ]:
print("="*70)
print("STEP 3: EIGENDECOMPOSITION")
print("="*70)

# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix_np)

# Sort by eigenvalue
idx = eigenvalues.argsort()[::-1]
eigenvalues_sorted = eigenvalues[idx]
eigenvectors_sorted = eigenvectors[:, idx]

# Explained variance ratio
explained_variance_ratio = eigenvalues_sorted / eigenvalues_sorted.sum()
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

print(f"\nTop 5 eigenvalues: {eigenvalues_sorted[:5]}")
print(f"Top 5 explained variance ratios: {explained_variance_ratio[:5]}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scree plot
axes[0].plot(range(1, 21), eigenvalues_sorted[:20], 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Principal Component', fontweight='bold')
axes[0].set_ylabel('Eigenvalue', fontweight='bold')
axes[0].set_title('Scree Plot', fontweight='bold')
axes[0].grid(alpha=0.3)

# Cumulative variance
axes[1].plot(range(1, 21), cumulative_variance_ratio[:20] * 100, 'ro-', linewidth=2, markersize=8)
axes[1].axhline(y=80, color='g', linestyle='--', label='80% variance')
axes[1].axhline(y=95, color='b', linestyle='--', label='95% variance')
axes[1].set_xlabel('Number of Components', fontweight='bold')
axes[1].set_ylabel('Cumulative Explained Variance (%)', fontweight='bold')
axes[1].set_title('Cumulative Variance Explained', fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

n_components_95 = np.argmax(cumulative_variance_ratio >= 0.95) + 1
print(f"\nComponents needed for 95% variance: {n_components_95}")

## 9. PCA Step 4: Projection

In [ ]:
print("="*70)
print("STEP 4: PROJECTION")
print("="*70)

n_components = 10
W = eigenvectors_sorted[:, :n_components]
Z_manual = X_scaled @ W

print(f"\nOriginal data shape: {X_scaled.shape}")
print(f"Projection matrix shape: {W.shape}")
print(f"Projected data shape: {Z_manual.shape}")
print(f"Variance explained: {cumulative_variance_ratio[n_components-1]:.4f}")

## 10. Complete PCA Workflow with Scikit-learn

In [ ]:
print("="*70)
print("COMPLETE PCA WORKFLOW (scikit-learn)")
print("="*70)

pca = PCA(n_components=n_components)
Z_sklearn = pca.fit_transform(X_scaled)

print(f"\nExplained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total variance explained: {pca.explained_variance_ratio_.sum():.4f}")
print(f"\nManual and sklearn results match? {np.allclose(np.abs(Z_manual), np.abs(Z_sklearn))}")

## 11. Visualize PCA Results

In [ ]:
# 2D projection
plt.figure(figsize=(10, 8))
scatter = plt.scatter(Z_sklearn[:, 0], Z_sklearn[:, 1], 
                     c=ehr_data['age'], cmap='viridis', alpha=0.6, s=50, edgecolor='black')
plt.colorbar(scatter, label='Age (years)')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)', fontweight='bold')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)', fontweight='bold')
plt.title('PCA: First Two Principal Components', fontweight='bold', fontsize=14)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 3D projection
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(Z_sklearn[:, 0], Z_sklearn[:, 1], Z_sklearn[:, 2],
                    c=ehr_data['age'], cmap='plasma', alpha=0.6, s=50, edgecolor='black')
ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})', fontweight='bold')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})', fontweight='bold')
ax.set_zlabel(f'PC3 ({pca.explained_variance_ratio_[2]:.1%})', fontweight='bold')
ax.set_title('PCA: 3D Projection', fontweight='bold', fontsize=14)
plt.colorbar(scatter, label='Age (years)', pad=0.1)
plt.tight_layout()
plt.show()

In [ ]:
# Component loadings
feature_names = pca_data.columns[:15]
loadings = pca.components_[:3, :15].T

plt.figure(figsize=(12, 6))
x = np.arange(len(feature_names))
width = 0.25

plt.bar(x - width, loadings[:, 0], width, label='PC1', alpha=0.8)
plt.bar(x, loadings[:, 1], width, label='PC2', alpha=0.8)
plt.bar(x + width, loadings[:, 2], width, label='PC3', alpha=0.8)

plt.xlabel('Features', fontweight='bold')
plt.ylabel('Loading', fontweight='bold')
plt.title('PCA Component Loadings', fontweight='bold', fontsize=14)
plt.xticks(x, [name[:10] for name in feature_names], rotation=45, ha='right')
plt.legend()
plt.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## Summary

### Key Takeaways:

1. **Healthcare Data** is heterogeneous, longitudinal, and high-dimensional with challenges in privacy, quality, and interoperability

2. **Missing Data** mechanisms (MCAR, MAR, MNAR) require different handling strategies - from simple imputation to advanced methods like MICE

3. **Data Artifacts** from technical and systemic sources impact model performance and require careful detection and mitigation

4. **PCA Four Steps:**
   - Step 1: Standardize features
   - Step 2: Compute covariance matrix
   - Step 3: Eigendecomposition
   - Step 4: Select and project components

5. **Best Practice:** Data quality determines AI success - invest time in understanding your data!